In [477]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import requests



In [555]:
import random
import math
global round
round=0

def number_to_grade(num):
    if num==1:
        return "mid1"
    elif num==2:
        return "mid2"
    else:
        return "mid3"
def number_to_subject(num):
    if num==1:
        return "chinese"
    elif num==2:
        return "math"
    else:
        return "english"

def distance(school1,school2):
    (i1,j1)=school1.location
    (i2,j2)=school2.location
    return abs(i1-i2)+abs(j1-j2)

class school:
    def __init__(self,i,i1,j1,classneed):
        self.id=i
        self.location=(i1,j1)
        self.need={"mid1":{'chinese':classneed[0],'math':classneed[1],
        'english':classneed[2]},"mid2":{'chinese':classneed[3],'math':classneed[4],
        'english':classneed[5]},"mid3":{'chinese':classneed[6],'math':classneed[7],
        'english':classneed[8]}}
        self.arrange={"mid1":{'chinese':0,'math':0,
        'english':0},"mid2":{'chinese':0,'math':0,
        'english':0},"mid3":{'chinese':0,'math':0,
        'english':0}}
        self.dif={"mid1":{'chinese':0,'math':0,
        'english':0},"mid2":{'chinese':0,'math':0,
        'english':0},"mid3":{'chinese':0,'math':0,
        'english':0}}
        self.teachers=[]
        self.arrange_need=0
    def get_info(self):
        print(self.id,self.need,self.arrange)
    def get_dif(self):
        a={"mid1":{'chinese':0,'math':0,
        'english':0},"mid2":{'chinese':0,'math':0,
        'english':0},"mid3":{'chinese':0,'math':0,
        'english':0}}
        for i in ["mid1","mid2","mid3"]:
            for j in ["chinese","math","english"]:
                a[i][j]=self.need[i][j]-self.arrange[i][j]
        self.dif=a     
    def check(self):
        b=0
        c=0
        for i in ["mid1","mid2","mid3"]:
            for j in ["chinese","math","english"]:
                b+=self.need[i][j]
                c+=self.arrange[i][j]
        self.arrange_need=c/b
        if b==c:
            return True
        else:
            return False
        
class teacher:
    def __init__(self,i,subject,school_choose,grade_choose):
        self.id=i
        self.school=[]
        self.grade=[]
        self.priority=random.randint(1,5)
        self.school_choose=school_choose
        self.subject=subject
        self.grade_choose=grade_choose
        self.working_time=0
    def get_info(self):
        print(self.id,self.school,self.working_time)
    def enroll_school(self,schools,grade,working_time):  
        #print(schools.arrange)
        #print(round)
        sub=number_to_subject(self.subject)
        schools.arrange[grade][sub]+=working_time
        if schools not in self.school:
            self.school.append(schools)
        schools.teachers.append(self)
        self.working_time+=working_time
        if grade not in self.grade:
            self.grade.append(grade)
   
def rank_teacher(teacher_list):
    a=sorted(teacher_list,key=lambda x:x.priority,reverse=True)
    return a

def get_avg_working_time(school_list,teacher_list):
    need_working_time={'chinese':0,'math':0,
        'english':0}
    count_teacher_number={'chinese':0,'math':0,
        'english':0}
    for school in school_list:
        need_working_time["chinese"]+=school.need["mid1"]["chinese"]+school.need["mid2"]["chinese"]+school.need["mid3"]["chinese"]
        need_working_time["math"]+=school.need["mid1"]["math"]+school.need["mid2"]["math"]+school.need["mid3"]["math"]
        need_working_time["english"]+=school.need["mid1"]["english"]+school.need["mid2"]["english"]+school.need["mid3"]["english"]
    for teacher in teacher_list:
        if teacher.subject==1:
            count_teacher_number['chinese']+=1
        elif teacher.subject==2:
            count_teacher_number['math']+=1
        else:
            count_teacher_number['english']+=1
    #print(need_working_time,count_teacher_number)
    return {'chinese':math.ceil(need_working_time["chinese"]/count_teacher_number['chinese']/3)*3,
           'math':math.ceil(need_working_time["math"]/count_teacher_number['math']/3)*3,
           'english':math.ceil(need_working_time["english"]/count_teacher_number['english']/3)*3}

In [556]:
def greedy(teacher_list,school_list,tolerance=0.3,school_first=True,grade_first=False,tolerance_time=1.3,tolerance_distance=4):
    global round
    def meet_tolerance(schools,teachers,working_time,grade):
        try:
            grade=number_to_grade(grade)
        except:
            grade=grade
        subject=number_to_subject(teachers.subject)
        schools.get_dif()
        arrange_space=schools.dif[grade][subject]
        #print(grade,subject,arrange_space,working_time)
        if arrange_space>working_time:
            teachers.enroll_school(schools,grade,working_time)
        elif abs(arrange_space-working_time)<tolerance*working_time:
            teachers.enroll_school(schools,grade,arrange_space)
        else:
            return 
    
    def first_round(): #校区志愿,学科志愿均满足  
        global round
        round=1
        for teachers in teacher_list:
            for schools in school_list:
                schools.get_dif()
                if schools.check():
                    break
                if schools.id==teachers.school_choose:
                    if teachers.working_time==0:
                        meet_tolerance(schools,teachers,avg_working_time[number_to_subject(teachers.subject)],teachers.grade_choose) 
    def second_round(): #优先校区，随机年级
        global round
        round=2
        for teachers in teacher_list:
            if teachers.working_time==0:
                for schools in school_list:
                    schools.get_dif()
                    if schools.check():
                        break
                    if schools.id==teachers.school_choose:
                        for grade in [1,2,3]:
                            if teachers.working_time==0:
                                meet_tolerance(schools,teachers,avg_working_time[number_to_subject(teachers.subject)],grade) 

    def third_round(): #优先学科，随机校区
        global round
        round=3
        for teachers in teacher_list:
            if teachers.working_time==0:
                for schools in school_list:
                    schools.get_dif()
                    if schools.check():
                        break
                    grade=teachers.grade_choose
                    if teachers.working_time==0:
                        meet_tolerance(schools,teachers,avg_working_time[number_to_subject(teachers.subject)],grade)
    def fourth_round(): #随机填充单学科
        global round
        round=4
        k=-3
        while True:
            k=k+3  
            still_no_arrange=False
            for teachers in teacher_list:
                if teachers.working_time==0:
                    for schools in school_list:
                        schools.get_dif()
                        if schools.check():
                            break
                        for grade in [1,2,3]:
                            if teachers.working_time==0:
                                meet_tolerance(schools,teachers,avg_working_time[number_to_subject(teachers.subject)]-3*k,grade)
                            if teachers.working_time==0:
                                still_no_arrange=True
            if not still_no_arrange:
                break                  
                
    def fifth_round(): #多学科+当前校区
        global round
        round=5
        for teachers in teacher_list:
            if teachers.working_time<avg_working_time[number_to_subject(teachers.subject)]:
                try:
                    schools=teachers.school[0]
                except:
                    schools=teachers.school_choose
                schools.get_dif()
                if schools.check():
                    break
                if schools.id==teachers.school_choose:
                    for grade in [1,2,3] :
                        dif=avg_working_time[number_to_subject(teachers.subject)]-teachers.working_time
                        meet_tolerance(schools,teachers,dif,grade)

    def sixth_round(teacher_list):#跨双校区
        teacher_list=sorted(teacher_list,key=lambda x:x.working_time,reverse=False)
        global round
        round=6
        for teachers in teacher_list:
            dif=avg_working_time[number_to_subject(teachers.subject)]-teachers.working_time
            if abs(dif)/avg_working_time[number_to_subject(teachers.subject)]<tolerance:
                break
            for schools in school_list:
                schools.get_dif()
                if schools.check():
                    break
                for grade in teachers.grade:                        
                    if distance(teachers.school[0],schools)<tolerance_distance and len(teachers.school)==1:
                        dif=avg_working_time[number_to_subject(teachers.subject)]-teachers.working_time
                        meet_tolerance(schools,teachers,dif,grade)     

    def seventh_round():#校区内填充所有
        global round
        round=7
        for schools in school_list:
            teachers_in_schools=schools.teachers
            teachers_in_schools=sorted(teachers_in_schools,key=lambda x:x.working_time,reverse=False)
            schools.get_dif()
            dif=schools.dif
            for i in ["mid1","mid2","mid3"]:
                for j in ["chinese","math",'english']:
                    while True:
                        if dif[i][j]==0:
                            break
                        for teachers in teachers_in_schools:
                            if number_to_subject(teachers.subject)==j:
                                teachers.enroll_school(schools,i,3)
                                schools.get_dif()
                                dif=schools.dif
                                break
                        teachers_in_schools=sorted(teachers_in_schools,key=lambda x:x.working_time,reverse=False)

                           
                          
    avg_working_time=get_avg_working_time(school_list,teacher_list)
    max_working_time=max(avg_working_time["chinese"],avg_working_time["math"],avg_working_time["english"])*tolerance_time
    teacher_list=rank_teacher(teacher_list)
    first_round()
    if school_first:
        second_round()
        third_round()
    if grade_first:
        third_round()
        second_round()
    fourth_round()
    fifth_round()
    sixth_round(teacher_list)
    seventh_round()
    #eighth_round()
        

In [572]:
N=100#school numbers
n=3000 #teacher numbers
teacher_list=[teacher(i,random.randint(1,3),random.randint(1,N),random.randint(1,3)) for i in range(n)]
school_list=[];index=1
for i in range(10):
    for j in range(10):
        school_list.append(school(index,i,j,[random.randint(10,100)*3 for h in range(9)]))
        index=index+1
# for i in teacher_list:
#     i.get_info()
# for i in school_list:
#     i.get_info()
get=get_avg_working_time(school_list,teacher_list)
print(get)
#print(school_list)

{'chinese': 51, 'math': 48, 'english': 51}


In [573]:
greedy(teacher_list,school_list,tolerance=0.35,school_first=True,grade_first=False,tolerance_time=1.35,tolerance_distance=4)


In [579]:
count_multi_grade=0
count_multi_school=0
count_chosen_school=0
count_chosen_grade=0
count_avg=0
get=get_avg_working_time(school_list,teacher_list)
for i in teacher_list:
    if len(i.grade)>1:
        count_multi_grade+=1
    if len(i.school)>1:
        count_multi_school+=1
    for j in i.school:
        if j.id==i.school_choose:
            count_chosen_school+=1
            
    if number_to_grade(i.grade_choose) in i.grade:
        count_chosen_grade+=1
    if abs(i.working_time-get[number_to_subject(i.subject)])<=9:
        count_avg+=1

print("multi grade: ",count_multi_grade/len(teacher_list))
print("multi school: ",count_multi_school/len(teacher_list))
print("chosen school: ",count_chosen_school/len(teacher_list))
print("chosen grade: ",count_chosen_grade/len(teacher_list))
print("around avg working time: ",count_avg/len(teacher_list))

multi grade:  0.17133333333333334
multi school:  0.004
chosen school:  0.7856666666666666
chosen grade:  0.8583333333333333
around avg working time:  0.9243333333333333


In [563]:
count=0
nnn=0
count2=0
count3=0
for i in teacher_list:
    print(number_to_subject(i.subject),number_to_grade(i.grade_choose),i.grade,i.working_time,i.school)
    if i.working_time!=0:
        count+=1
    nnn+=1
    if number_to_grade(i.grade_choose) not in i.grade:
        if i.grade!=[]:
            count2+=1
print(count/nnn)
print(count2/nnn)

for i in school_list:
    print(i.dif)
    print(i.check())
    print(i.arrange_need)

    

#     print(i,"\n")
#     print(i.arrange)
#     print(i.need)
#     i.get_dif()


math mid3 ['mid3', 'mid2'] 51 [<__main__.school object at 0x1181de6d8>]
math mid1 ['mid1', 'mid2'] 51 [<__main__.school object at 0x1181dee48>]
chinese mid2 ['mid2'] 54 [<__main__.school object at 0x11732a0b8>]
math mid2 ['mid2'] 48 [<__main__.school object at 0x1181de9b0>]
math mid1 ['mid1'] 48 [<__main__.school object at 0x1181de9b0>]
chinese mid3 ['mid3'] 54 [<__main__.school object at 0x1181deb70>]
chinese mid1 ['mid1'] 54 [<__main__.school object at 0x1170ac588>]
math mid2 ['mid2'] 48 [<__main__.school object at 0x1170ac828>]
math mid2 ['mid2'] 48 [<__main__.school object at 0x1170acf60>]
math mid3 ['mid1'] 9 [<__main__.school object at 0x1170ace48>]
math mid1 ['mid1'] 48 [<__main__.school object at 0x1181de400>]
chinese mid1 ['mid2'] 33 [<__main__.school object at 0x1181de240>]
chinese mid2 ['mid2'] 18 [<__main__.school object at 0x1170ac828>]
chinese mid3 ['mid3'] 54 [<__main__.school object at 0x1181de240>]
math mid1 ['mid3'] 12 [<__main__.school object at 0x1170ac898>]
chinese